In [2]:
import torch

import configs
from transformer.model import Transformer
from paths import PROJECT_ROOT

/home/taot/github/grokking-lab/src
/home/taot/github/grokking-lab


In [3]:
run_path = PROJECT_ROOT / "runs/transformer/one_layer/20260217_211250"

In [4]:
cfg = configs.load_config(run_path / "config.yaml")

model = Transformer(
    vocab_size=cfg.vocab_size, d=cfg.d, n_layers=cfg.n_layers, h=cfg.h
).to(device=cfg.device)

In [5]:
model.load_state_dict(torch.load(run_path / "checkpoint.pt", map_location=cfg.device))

<All keys matched successfully>

In [6]:
model.linear.weight.shape

torch.Size([113, 128])

In [ ]:
linear_weight = model.linear.weight.detach()
linear_weight_fft_per_column = torch.fft.fft(linear_weight, dim=0)
linear_weight_fft_per_column.shape

In [ ]:
linear_weight_fft_per_column

In [ ]:
linear_weight_fft_abs = linear_weight_fft_per_column.abs()
linear_weight_fft_abs

In [ ]:
linear_weight_fft_abs_filtered = torch.where(
    linear_weight_fft_abs < 10,
    torch.zeros_like(linear_weight_fft_abs),
    linear_weight_fft_abs,
)
linear_weight_fft_abs_filtered

# Periodicity

In [7]:
embed = model.embedding.weight.detach()

In [13]:
embed.shape

torch.Size([113, 128])

In [10]:
torch.linalg.norm(torch.fft.fft(embed[10], dim=0))

tensor(75.1818, device='cuda:0')

In [12]:
torch.fft.fft(embed, dim=0)

tensor([[ 8.1515+0.0000j, -4.2183+0.0000j, -0.9846+0.0000j,  ...,
         -7.0398+0.0000j,  4.0386+0.0000j, -4.8046+0.0000j],
        [-7.7546-8.6809j,  1.8619+2.0774j, -3.5898+0.7212j,  ...,
         -1.3816+5.3237j, -1.1982-2.6815j, -1.3153-2.0606j],
        [ 1.1555+2.9802j, -5.2664+2.3312j, -2.6978-0.1577j,  ...,
          3.5293+3.9703j,  4.0301-1.5023j, -1.0872+2.1251j],
        ...,
        [ 2.7697+0.6644j,  0.1233+0.9397j,  2.2733-2.4693j,  ...,
         -2.0840-0.2775j, -1.1704-2.7260j, -3.7489-2.1112j],
        [ 1.1555-2.9802j, -5.2664-2.3312j, -2.6978+0.1577j,  ...,
          3.5293-3.9703j,  4.0301+1.5023j, -1.0872-2.1251j],
        [-7.7546+8.6809j,  1.8619-2.0774j, -3.5898-0.7212j,  ...,
         -1.3816-5.3237j, -1.1982+2.6815j, -1.3153+2.0606j]], device='cuda:0')

In [14]:
E = model.embedding.weight.detach()          # [p, d]
E_fft = torch.fft.fft(E, dim=0)              # 沿 token 维做 FFT
E_fft_norm = torch.linalg.norm(E_fft, dim=1) # 每个频率的L2范数
E_fft_norm[:57]                              # p=113时看前56/57个（共轭对称）

tensor([ 62.0091,  74.5033,  47.2718,  50.6563,  51.6046,  60.3256,  89.6413,
        121.3770,  56.2937,  46.1682,  47.5412,  69.7170,  52.8863,  49.6471,
         58.3695,  49.5583,  47.5978,  49.2456,  51.2726,  46.6369,  50.1106,
         64.6495,  53.6478,  48.0652,  49.0819, 134.8198,  51.2420, 127.8213,
         45.5175,  49.1781,  54.8412,  52.2483,  54.5371,  50.9778,  47.9051,
         45.4582,  47.9624,  50.4367,  55.1305,  52.7403,  48.9116,  47.1435,
         58.7810,  56.3584,  46.7414,  49.6810, 117.3979,  58.2533,  47.2635,
         59.5905,  65.4156, 116.7754,  48.1603,  47.9888,  70.3255,  54.2862,
         50.4366], device='cuda:0')